# 로컬에서 훈련 하기
- 이 노트북은 로컬 (현재 머신) 에서 Hugging Face Accelerator + PyTorch FSDP 로 파인 튜닝 합니다.

## 1. 환경 셋업

### Hugging Face Token 입력
- [중요] HF Key 가 노출이 안되도록 조심하세요.

In [1]:
import os

def set_hf_key_env_vars(hf_key_name, key_val):
    os.environ[hf_key_name] = key_val

def get_hf_key_env_vars(hf_key_name):
    HF_key_value = os.environ.get(hf_key_name)

    return HF_key_value


is_sagemaker_notebook = True
if is_sagemaker_notebook:
    hf_key_name = "HF_KEY"
    key_val = "<Type Your HF Key>"
    set_hf_key_env_vars(hf_key_name, key_val)
    HF_TOKEN = get_hf_key_env_vars(hf_key_name)
else: # VS Code
    from dotenv import load_dotenv
    HF_TOKEN = os.getenv('HF_TOKEN')


# Log in to HF
!huggingface-cli login --token {HF_TOKEN}


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/ec2-user/SageMaker/.cache/token
Login successful


### 저장된 변수 로딩

In [2]:
%store -r data_folder
%store -r train_data_json 
%store -r validation_data_json 
%store -r test_data_json 
%store -r full_train_data_json 
%store -r full_validation_data_json 
%store -r full_test_data_json


print("data_folder: ", data_folder)
print("train_data_json: ", train_data_json)
print("validation_data_json: ", validation_data_json)
print("test_data_json: ", test_data_json)
print("full_train_data_json: ", full_train_data_json)
print("full_validation_data_json: ", full_validation_data_json)
print("full_test_data_json: ", full_test_data_json)

data_folder:  ../data/naver-news-summarization-ko
train_data_json:  ../data/naver-news-summarization-ko/train/train_dataset.json
validation_data_json:  ../data/naver-news-summarization-ko/validation/validation_dataset.json
test_data_json:  ../data/naver-news-summarization-ko/test/test_dataset.json
full_train_data_json:  ../data/naver-news-summarization-ko/full_train/train_dataset.json
full_validation_data_json:  ../data/naver-news-summarization-ko/full_validation/validation_dataset.json
full_test_data_json:  ../data/naver-news-summarization-ko/full_test/test_dataset.json


In [3]:
import torch
import time
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

from datasets import Dataset, load_dataset
from datasets import load_dataset, load_metric
from transformers import pipeline, set_seed
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import warnings
warnings.filterwarnings("ignore")

/home/ec2-user/anaconda3/envs/llama3-py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ec2-user/anaconda3/envs/llama3-py310/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


## 2. 베이스 모델 준비

In [4]:
model_id = "meta-llama/Meta-Llama-3-8B"
output_dir = "/home/ec2-user/SageMaker/models/llama-3-8b-naver-news"

### Config YAML 파일 생성

In [5]:
%%writefile accelerator_config/local_llama_3_8b_fsdp_qlora.yaml
# script parameters
model_id:  "meta-llama/Meta-Llama-3-8B" # Hugging Face model id
###########################
# small samples for Debug
###########################
train_dataset_path: "../data/naver-news-summarization-ko/train"                      # path to dataset
validation_dataset_path: "../data/naver-news-summarization-ko/validation"                      # path to dataset
test_dataset_path: "../data/naver-news-summarization-ko/test"                      # path to dataset
per_device_train_batch_size: 1         # batch size per device during training
per_device_eval_batch_size: 1          # batch size for evaluation
gradient_accumulation_steps: 2         # number of steps before performing a backward/update pass
###########################
# large samples for evaluation
###########################
# train_dataset_path: "../data/naver-news-summarization-ko/full_train"                      # path to dataset
# validation_dataset_path: "../data/naver-news-summarization-ko/full_validation"                      # path to dataset
# test_dataset_path: "../data/naver-news-summarization-ko/full_test"                      # path to dataset
# per_device_train_batch_size: 16         # batch size per device during training
# per_device_eval_batch_size: 1          # batch size for evaluation
# gradient_accumulation_steps: 2         # number of steps before performing a backward/update pass
###########################
max_seq_len:  2048              # max sequence length for model and packing of the dataset
# training parameters
output_dir: "/home/ec2-user/SageMaker/models/llama-3-8b-naver-news" # Temporary output directory for model checkpoints
report_to: "tensorboard"               # report metrics to tensorboard
logging_dir: "/home/ec2-user/SageMaker/logs/llama-3-8b-naver-news" # log folder for tensorboard
learning_rate: 0.0002                  # learning rate 2e-4
lr_scheduler_type: "constant"          # learning rate scheduler
num_train_epochs: 1                    # number of training epochs
optim: adamw_torch                     # use torch adamw optimizer
logging_steps: 10                      # log every 10 steps
save_strategy: epoch                   # save checkpoint every epoch
evaluation_strategy: epoch             # evaluate every epoch
max_grad_norm: 0.3                     # max gradient norm
warmup_ratio: 0.03                     # warmup ratio
bf16: true                             # use bfloat16 precision
tf32: true                             # use tf32 precision
gradient_checkpointing: true           # use gradient checkpointing to save memory
# FSDP parameters: https://huggingface.co/docs/transformers/main/en/fsdp
fsdp: "full_shard auto_wrap offload" # remove offload if enough GPU memory
fsdp_config:
  backward_prefetch: "backward_pre"
  forward_prefetch: "false"
  use_orig_params: "false"

Overwriting accelerator_config/local_llama_3_8b_fsdp_qlora.yaml


## 3. 훈련 Script 실행

아래는 Hugging Face 의 Accelerator 를 이용한 PyTorch FSDP 를 실행하기 위한 명령어 입니다.
- 현재 머신에 4개의 GPU 가 있는 경우 입니다. GPU 가 1개 이면 nproc_per_node=1 로 수정해서 실행 하세요. 

```
!ACCELERATE_USE_FSDP=1 FSDP_CPU_RAM_EFFICIENT_LOADING=1 torchrun --nproc_per_node=4 \
../../scripts/local_run_fsdp_qlora.py \
--config config_folder_name/local_llama_3_8b_fsdp_qlora.yaml
```
- 참고
    - Launching your 🤗 Accelerate scripts, [Link](https://huggingface.co/docs/accelerate/en/basic_tutorials/launch)

In [6]:
import os
config_folder_name = "accelerator_config"
os.makedirs(config_folder_name, exist_ok=True)

### Hugging Face  Accelerator 에 제공할 config.yaml 입니다.

In [7]:
!ACCELERATE_USE_FSDP=1 FSDP_CPU_RAM_EFFICIENT_LOADING=1 torchrun --nproc_per_node=1 \
../../scripts/local_run_fsdp_qlora.py \
--config accelerator_config/local_llama_3_8b_fsdp_qlora.yaml

/home/ec2-user/anaconda3/envs/llama3-py310/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
## script_args: 
 ScriptArguments(train_dataset_path='../data/naver-news-summarization-ko/train', validation_dataset_path='../data/naver-news-summarization-ko/validation', model_id='meta-llama/Meta-Llama-3-8B', max_seq_length=512)
## training_args: 
 TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'gradient_accumulation_kwargs': None},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=True,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,

## 4. 베이스 모델과 훈련된 모델 머지

In [8]:
model_id, output_dir

('meta-llama/Meta-Llama-3-8B',
 '/home/ec2-user/SageMaker/models/llama-3-8b-naver-news')

### 모델 머지 및 로컬에 저장
- 약 2분 걸림

In [9]:
from peft import AutoPeftModelForCausalLM
import torch

# Load PEFT model on CPU

model = AutoPeftModelForCausalLM.from_pretrained(
    output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)  
# Merge LoRA and base model and save
merged_model = model.merge_and_unload()
merged_model.save_pretrained(output_dir,safe_serialization=True, max_shard_size="2GB")

Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.09it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
model.device, merged_model.device

(device(type='cpu'), device(type='cpu'))

### 머지된 모델 로딩

In [11]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer 


# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  pretrained_model_name_or_path = output_dir,
  torch_dtype=torch.float16,
  quantization_config= {"load_in_4bit": True},
  device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.47s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 5. 추론

### 테스트 데이터 셋 로딩

In [12]:
from datasets import load_dataset 
from random import randint

def get_message_from_dataset(sample_dataset_json_file):
    # Load our test dataset
    full_test_dataset = load_dataset("json", data_files=sample_dataset_json_file, split="train")

    # Test on sample 
    rand_idx = randint(0, len(full_test_dataset)-1)
    rand_idx = 75
    print("rand_idx: ", rand_idx)
    messages = full_test_dataset[rand_idx]["messages"][:2]
    # messages = test_dataset[rand_idx]["text"][:2]
    print("messages: \n", messages)

    return messages, full_test_dataset, rand_idx

messages, full_test_dataset, rand_idx = get_message_from_dataset(sample_dataset_json_file = full_test_data_json)    

Generating train split: 2740 examples [00:00, 99615.95 examples/s]

rand_idx:  75
messages: 
 [{'content': 'You are an AI assistant specialized in news articles.Your role is to provide accurate summaries and insights in Korean. Please analyze the given text and provide concise, informative summaries that highlight the key goals and findings.', 'role': 'system'}, {'content': 'Please summarize the goals for journalist in this text:\n\n산업현장에서 세종텔레콤의 스마트 안전 플랫폼 솔루션 을 활용하고 있다. 세종텔레콤 제공 세종텔레콤은 태영건설에 중대재해처벌법 대응에 최적화된 스마트 안전 플랫폼 솔루션 납품 계약을 체결했다고 4일 밝혔다. 우선 태영건설 전국 산업현장에 스마트 안전 솔루션 500대 납품 계약을 체결했고 추가 구축을 협의 중이다. 지난 1월 본격 시행된 중대재해처벌법은 산업현장에서 인명사고 발생 시 경영진이나 법인에게 책임을 물을 수 있도록 규정한 법이다. 해당 법은 안전 보건 관련 관리상의 조치 구축을 의무화하고 있으나 기업의 한정적 자원과 부족한 인력 문제로 어려움을 겪고 있다. 세종텔레콤의 스마트 안전 플랫폼 솔루션은 출입관리부터 CCTV 가스탐지 각종 센서 등을 하나로 통합해 현장을 종합 관리할 수 있다. LBS 위치기반 IoT 사물인터넷 등 스마트 기술을 융합했다. 안전 관리 담당자는 각 현장마다 설치된 카메라 및 CCTV 개소별 센서와 통신 인프라를 통해 현장 정보를 실시간으로 확인하고 비상 상황 시에는 전체 현장 또는 해당 구역 상황실 시스템이나 모바일로 근로자에게 안전 조치사항을 지시할 수 있다. 이와 함께 타워크레인에 설치한 360도 카메라를 통해 현장의 불안전 요소를 발견하면 관계자에게 알림을 보낼 수 있다. 지하

### 추론

In [13]:

def generate_response(messages, model, tokenizer, full_test_dataset, rand_idx):
    input_ids = tokenizer.apply_chat_template(messages,add_generation_prompt=True,return_tensors="pt").to(model.device)
    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id= tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]

    print(f"**Query:**\n{full_test_dataset[rand_idx]['messages'][1]['content']}\n")
    # print(f"**Query:**\n{test_dataset[rand_idx]['text'][1]['content']}\n")
    # print(f"**Original Answer:**\n{test_dataset[rand_idx]['text'][2]['content']}\n")
    print(f"**Original Answer:**\n{full_test_dataset[rand_idx]['messages'][2]['content']}\n")
    print(f"**Generated Answer:**\n{tokenizer.decode(response,skip_special_tokens=True)}")

generate_response(messages, model, tokenizer, full_test_dataset, rand_idx)    



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


**Query:**
Please summarize the goals for journalist in this text:

산업현장에서 세종텔레콤의 스마트 안전 플랫폼 솔루션 을 활용하고 있다. 세종텔레콤 제공 세종텔레콤은 태영건설에 중대재해처벌법 대응에 최적화된 스마트 안전 플랫폼 솔루션 납품 계약을 체결했다고 4일 밝혔다. 우선 태영건설 전국 산업현장에 스마트 안전 솔루션 500대 납품 계약을 체결했고 추가 구축을 협의 중이다. 지난 1월 본격 시행된 중대재해처벌법은 산업현장에서 인명사고 발생 시 경영진이나 법인에게 책임을 물을 수 있도록 규정한 법이다. 해당 법은 안전 보건 관련 관리상의 조치 구축을 의무화하고 있으나 기업의 한정적 자원과 부족한 인력 문제로 어려움을 겪고 있다. 세종텔레콤의 스마트 안전 플랫폼 솔루션은 출입관리부터 CCTV 가스탐지 각종 센서 등을 하나로 통합해 현장을 종합 관리할 수 있다. LBS 위치기반 IoT 사물인터넷 등 스마트 기술을 융합했다. 안전 관리 담당자는 각 현장마다 설치된 카메라 및 CCTV 개소별 센서와 통신 인프라를 통해 현장 정보를 실시간으로 확인하고 비상 상황 시에는 전체 현장 또는 해당 구역 상황실 시스템이나 모바일로 근로자에게 안전 조치사항을 지시할 수 있다. 이와 함께 타워크레인에 설치한 360도 카메라를 통해 현장의 불안전 요소를 발견하면 관계자에게 알림을 보낼 수 있다. 지하 작업에서는 이동형 스마트 영상 장비로 안전 사각지대를 살필 수 있고 밀폐된 작업 공간에서는 가스 센서와 신호등형 전광판을 설치해 실시간으로 스마트 상황판에 가스 농도를 전송한다. 유해가스가 허용 농도를 초과하면 현장에서 환기 시스템이 자동으로 작동한다. 현장 내 추락 사고가 발생할 수 있는 개구부에 부착한 센서는 개구부가 비정상적으로 개폐됐을 때 경고음을 보내 위험상황을 알린다. 강효상 세종텔레콤 통신사업본부장은 중대재해처벌법 시행에 따른 건설 현장의 안전사고 예방을 위한 최적의 솔루션이 요구되고 있는 시점에서 스마트 안전 솔루션 

### 할당된 CUDA memory를 Release

In [14]:
del model
del tokenizer
torch.cuda.empty_cache()